**Nota:** Este cuaderno está diseñado para **Google Colab**.

Si ves el logotipo de Colab <span style='vertical-align:bottom;'><img src='https://colab.research.google.com/img/colab_favicon_256px.png' width='40' alt='Logotipo de Colab'></span> en la esquina superior izquierda, ¡ya está todo listo! Por favor, **continúa**.

Si no ves el logotipo (por ejemplo, estás en GitHub), haz clic en el botón de abajo para abrirlo en el entorno correcto:

[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mparrott-at-wiris/aimodelshare/blob/master/notebooks/justice_and_equity_advance_notebook_es.ipynb)

# **Reto Avanzado de Justicia y Equidad: Construir y Enviar Modelos Personalizados**

Bienvenido a la **Vía Avanzada** del Reto de Justicia de Ética en Juego (Ethics at Play).

**¿Para quién es esto?**
Este cuaderno está diseñado para participantes con experiencia en Python (por ejemplo, Scikit-Learn, TensorFlow, PyTorch). En lugar de usar las aplicaciones gamificadas, construirás, entrenarás y enviarás tus propios modelos de aprendizaje automático directamente a la tabla de clasificación de la competencia.

**El Objetivo:**
Entrenar un modelo para predecir el riesgo de reincidencia (la probabilidad de volver a delinquir) utilizando el conjunto de datos COMPAS, equilibrando la precisión y la equidad.

## 🚀 **Guía de Inicio Rápido**

Para participar en el reto, completa estos 5 pasos:

1.  **Instalar Bibliotecas:** Ejecuta la celda de configuración para instalar `aimodelshare`.
2.  **Obtener los Datos:** Ejecuta la celda de carga de datos para recuperar los datos de entrenamiento y prueba pre-divididos.
3.  **Entrenar tu Modelo:** Usa el ejemplo de Pipeline de Scikit-Learn proporcionado o escribe tu propio código de entrenamiento personalizado.
4.  **Conectar:** Vincula este cuaderno a la Tabla de Clasificación del Reto de Justicia.
5.  **Enviar:** Envía tus predicciones a la tabla de clasificación para ver tu puntuación.

**¿Listo? Haz clic en el botón de reproducción ▶ en la primera celda de abajo para comenzar.**

---
# **Paso 1: Instalación**

Necesitamos instalar la biblioteca `aimodelshare` para conectarnos al backend de la competencia.

In [ ]:
# Instalar la biblioteca aimodelshare
print("Instalando las bibliotecas necesarias...")
!pip install aimodelshare --upgrade -q --no-warn-script-location > /dev/null 2>&1
print("✅ ¡Instalación completada!")

---
# **Paso 2: Cargar Datos**

Cargaremos los datos de entrenamiento y prueba directamente desde las URL oficiales de la competencia.

* **X_train:** Características para el entrenamiento.
* **y_train:** Etiquetas objetivo (¿ocurrió reincidencia?) para el entrenamiento.
* **X_test:** Características para la prueba (generarás predicciones sobre esto).

In [ ]:
import pandas as pd

# 1. Cargar Datos desde las URL
X_train = pd.read_csv("https://raw.githubusercontent.com/AIModelShare/aimodelshare_tutorials/refs/heads/main/datasets/ethicsatplay/X_train.csv")
X_test = pd.read_csv("https://raw.githubusercontent.com/AIModelShare/aimodelshare_tutorials/refs/heads/main/datasets/ethicsatplay/X_test.csv")
y_train_labels = pd.read_csv("https://raw.githubusercontent.com/AIModelShare/aimodelshare_tutorials/refs/heads/main/datasets/ethicsatplay/y_train.csv")

# Asegurar que y_train es una Serie 1D (requerido para sklearn)
y_train = y_train_labels.squeeze()

# 2. Definir Listas de Características para el Pipeline
# Estas listas coinciden con las columnas presentes en X_train y X_test
ALL_NUMERIC_COLS = ["juv_fel_count", "juv_misd_count", "juv_other_count", "days_b_screening_arrest", "age", "length_of_stay", "priors_count"]
ALL_CATEGORICAL_COLS = ["race", "sex", "c_charge_degree", "c_charge_desc"]

print("✅ ¡Datos cargados exitosamente!")
print(f"Forma de los datos de entrenamiento: {X_train.shape}")
print(f"Forma de los datos de prueba: {X_test.shape}")
print("\nPrimeras 5 filas de los datos de entrenamiento:")
X_train.head()

---
# **Paso 3: Entrenar el Modelo con la Biblioteca Preferida (Sklearn, Tensorflow, Pytorch, Etc.)**

Usaremos un **Pipeline de Scikit-Learn** para agilizar el preprocesamiento y el modelado.

Este pipeline:
1.  **Imputará Valores Faltantes** (Rellena NaNs con la mediana para números, el más frecuente para categorías).
2.  **Codificará One-Hot** las columnas categóricas (Raza, Sexo, Grado del Cargo, Descripción del Cargo).
3.  **Escalará** las columnas numéricas (Edad, Antecedentes, Duración de la Estancia, etc.).
4.  **Entrenará** un clasificador de Regresión Logística.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# 1. Definir grupos de características usando constantes del Paso 2
# Las características numéricas serán imputadas y escaladas
numeric_features = ALL_NUMERIC_COLS

# Las características categóricas serán imputadas y codificadas con One-Hot
categorical_features = ALL_CATEGORICAL_COLS

# 2. Definir Transformadores con Imputación
# Numérico: Imputar valores faltantes con la mediana, luego escalar
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categórico: Imputar valores faltantes con el valor más frecuente, luego codificar con one-hot
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# 3. Crear Preprocesador usando los transformadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 4. Crear Pipeline (Preprocesador + Modelo)
# Puedes reemplazar LogisticRegression con cualquier otro modelo de sklearn (ej. RandomForestClassifier)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# 5. Entrenar el pipeline
pipeline.fit(X_train, y_train)

# 6. Generar predicciones en el conjunto de prueba
predictions = pipeline.predict(X_test)

print(f"✅ ¡Modelo Entrenado! Predicciones generadas para {len(predictions)} muestras.")
print("Procede al siguiente paso para enviar tus predicciones a la tabla de clasificación.")

---
# **Paso 4: Conectar a la Tabla de Clasificación**

Este paso conecta tu cuaderno al backend específico para el Reto de Justicia y Equidad.

*Nota: Se te pedirá que ingreses un nombre de usuario y contraseña. Si no tienes uno, deberás crear uno en [modelshare.ai](https://www.modelshare.ai)

In [ ]:
from aimodelshare.aws import set_credentials
from aimodelshare.playground import Competition
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# La URL específica del Model Playground para el Reto de Justicia
my_playground_url = "https://cf3wdpkg0d.execute-api.us-east-1.amazonaws.com/prod/m"

# Configura tus credenciales (aparecerá una ventana emergente)
set_credentials(apiurl=my_playground_url)

# Generar tu token de acceso a la sesión
token=os.getenv("AWS_TOKEN")

# Conectar a la competencia
playground = Competition(my_playground_url)

---
# **Paso 5: Enviar y Verificar Resultados**

Envía tus predicciones a la tabla de clasificación.

In [ ]:
# 1. Enviar tus predicciones
# Nota: Pasamos None para el modelo y preprocesador porque solo estamos enviando predicciones para evaluación
playground.submit_model(
    model=None,
    preprocessor=None,
    prediction_submission=predictions, token=token,
    input_dict={
        "Team": "The Ethical Explorers", # Cambia el nombre del equipo manualmente si es necesario.
        "description": "Logistic Regression with Sklearn Pipeline",
        "tags": "sklearn, logistic_regression, advanced_pathway, pipeline"
    }
)

print("✅ ¡Predicciones enviadas exitosamente!")

# 2. Verificar la tabla de clasificación
print("Cargando tabla de clasificación...")
leaderboard = playground.get_leaderboard()
playground.stylize_leaderboard(leaderboard)